## 🔹 ColumnTransformer in Scikit-learn

**Purpose:** Apply different preprocessing steps to different columns of a dataset in a single, unified step. Useful for **numerical, categorical, ordinal features and missing values**.  

---

### 1️⃣ Why use ColumnTransformer?
- Handles **mixed types of data** in one step.  
- Avoids manually applying **One-Hot, Ordinal Encoding, Imputation, or Scaling** column by column.  
- Integrates smoothly with **pipelines** for modeling.  

---

### 2️⃣ Basic Syntax

```python
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'salary']),            # Numerical columns
        ('cat', OneHotEncoder(), ['gender', 'city']),           # Categorical columns
        ('ord', OrdinalEncoder(), ['education']),               # Ordinal columns
        ('impute', SimpleImputer(strategy='mean'), ['experience'])  # Missing values
    ]
)


### Examples

In [1]:
import numpy as np
import pandas as pd 

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')


In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2, random_state=0)  

In [9]:
x_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


### 1. Without Column Transformer

In [15]:
# Simple Imputer
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.fit_transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [17]:
# Ordinal Encoder -> Cough
oe= OrdinalEncoder(categories = [['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])
x_train_cough.shape

(80, 1)

In [22]:
# One Hot Encoder -> gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False , dtype=np.int32)   
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])
x_test_gender_city = ohe.fit_transform(x_test[['gender', 'city']])

x_train_gender_city.shape

(80, 4)

In [23]:
#Extracting Age column
x_train_age = x_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
x_test_age = x_test.drop(columns=['gender', 'fever', 'cough', 'city']).values 

x_train_age.shape

(80, 1)

In [25]:
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_cough, x_train_gender_city), axis=1)
# also test data
x_test_transformed = np.concatenate((x_test_age, x_test_fever, x_test_cough, x_test_gender_city), axis=1)
x_train_transformed.shape

(80, 7)

### Using Column Transform

In [27]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2',OrdinalEncoder(categories = [['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(drop='first', sparse_output=False , dtype=np.int32), ['gender', 'city'])
    ], remainder = 'passthrough')

In [33]:
ct.fit_transform(x_train).shape

(80, 7)

In [34]:
ct.transform(x_test).shape

(20, 7)